In [3]:
import _sqlite3 as sq
import requests
import re
from bs4 import BeautifulSoup

PAGE_SITE_END = 8
URL = "https://auto.drom.ru/all"
reccount = 0
sqllist = []

conn = sq.connect("dromru.db")
cursor = conn.cursor()

cursor.execute("drop table if exists car_records")
cursor.execute("drop table if exists car_names")

cursor.execute("create table car_names (" + 
               "name_id integer primary key," +
               "name text not null)")

cursor.execute("create table car_records (" + 
               "car_id integer primary key," +
               "name_id integer not null," +
               "year integer not null," +
               "power integer not null," +
               "drive text not null," +
               "run integer not null," +
               "city text not null," +
               "price integer not null," +
               "foreign key (name_id) references car_names (name_id) )")

conn.commit()

for page in range(1, PAGE_SITE_END + 1):

    content = requests.get(URL + "/page" + str(page) + "/")
    content_page = BeautifulSoup(content.text, 'html.parser')
    var = content_page.find_all("a", "css-4zflqt e1huvdhj1")
    for datastr in var:
        yearstr = datastr.find("div", "css-16kqa8y e3f4v4l2").text[-4:] if datastr.find("div", "css-16kqa8y e3f4v4l2") is not None else "0"   
        citystr = datastr.find("span", "css-1488ad e162wx9x0").text if datastr.find("span", "css-1488ad e162wx9x0") is not None else "нет"   
        pricestr = datastr.find("span", "css-46itwz e162wx9x0").text.replace("\xa0", "") if datastr.find("span", "css-46itwz e162wx9x0") is not None else "0р"
        pricestr = pricestr[:len(pricestr) - 1]
        
        detpageURL = datastr['href']
        
        detpage = requests.get(detpageURL)
        detpafe_content = BeautifulSoup(detpage.text, 'html.parser')
        brandname = detpafe_content.find("div", "hjz6860").find("a").text if detpafe_content.find("div", "hjz6860") is not None else "нет"
        powerstr = detpafe_content.find("span", "css-9g0qum e162wx9x0").text if detpafe_content.find("span", "css-9g0qum e162wx9x0") is not None else "0"
        powerstr = powerstr[:powerstr.find("\xa0")] if detpafe_content.find("span", "css-9g0qum e162wx9x0") is not None else "0"
        drivestr = detpafe_content.find_all("td", "css-1la7f7n ezjvm5n0")[3].text
        runstr = detpafe_content.find("span", "css-1osyw3j ei6iaw00").text if detpafe_content.find("span", "css-1osyw3j ei6iaw00") is not None else "0"
        
        if detpafe_content.find("span", "css-1osyw3j ei6iaw00") is not None:
            if runstr.find("\xa0", runstr.find("\xa0")+1) > 0:
                runstr = runstr[:runstr.find("\xa0", runstr.find("\xa0")+1)].replace("\xa0", "")
            else:
                runstr = runstr[:runstr.find("\xa0")].replace("\xa0", "")        
        else:
            runstr = "0"
            
        
        cursor.execute("select name_id from car_names where name=\"" + brandname + "\"")
        brandFound = cursor.fetchall()
        
        if len(brandFound) == 0:
            cursor.execute("insert into car_names (name) values (\"" + brandname + "\")")
            conn.commit()
            
            cursor.execute("select name_id from car_names where name=\"" + brandname + "\"")
            brandFound = cursor.fetchall()
        
        nameIdstr = str(brandFound[0][0])
        reccount = reccount + 1
        sqllist.append("insert into car_records (name_id, year, power, drive, run, city, price) " +
                      "values ("+ nameIdstr +", " +
                      yearstr + ", " +
                      powerstr + ", " +
                      "\"" + drivestr + "\", " +
                      runstr + ", " +
                      "\"" + citystr + "\", " +
                      pricestr + ")")
        

        print(reccount, "insert into car_records (name_id, year, power, drive, run, city, price) " +
                        "values ("+ nameIdstr +", " + 
                                yearstr + ", " +
                                powerstr + ", " + 
                                "\"" + drivestr + "\", " + 
                                runstr + ", " + 
                                "\"" + citystr + "\", " +
                                pricestr + ")")
        
for sqlrow in sqllist:
    cursor.execute(sqlrow)

cursor.close()
conn.commit()        
conn.close()



1 insert into car_records (name_id, year, power, drive, run, city, price) values (1, 2011, 88, "передний", 167000, "Уссурийск", 920000)
2 insert into car_records (name_id, year, power, drive, run, city, price) values (2, 2007, 136, "передний", 290000, "Партизанск", 1000000)
3 insert into car_records (name_id, year, power, drive, run, city, price) values (3, 2009, 136, "передний", 136000, "Владивосток", 1135000)
4 insert into car_records (name_id, year, power, drive, run, city, price) values (4, 2018, 91, "передний", 99000, "Владивосток", 898000)
5 insert into car_records (name_id, year, power, drive, run, city, price) values (5, 2020, 182, "4WD", 63000, "Япония → Владивосток", 3855000)
6 insert into car_records (name_id, year, power, drive, run, city, price) values (5, 2019, 152, "4WD", 99000, "Япония → Владивосток", 4285000)
7 insert into car_records (name_id, year, power, drive, run, city, price) values (5, 2020, 152, "4WD", 120000, "Япония → Владивосток", 4125000)
8 insert into car_

In [13]:
conn = sq.connect("dromru.db")
cursor = conn.cursor()

cursor.execute("select city, round(avg(price), 2) as avgPrice, count(name_id) as number, round(avg(run),2) as avgRun from car_records group by city order by number desc limit 5")
rows = cursor.fetchall()

for row in rows:
    print(row)
    
cursor.close()
conn.close()


('Владивосток', 1794090.9, 77, 116863.52)
('Япония → Владивосток', 3104952.38, 42, 54452.38)
('Уссурийск', 804588.12, 17, 196827.29)
('Находка', 2394142.86, 7, 172619.0)
('Артём', 868333.33, 6, 187166.5)


In [15]:
conn = sq.connect("dromru.db")
cursor = conn.cursor()

cursor.execute("select cn.name, cr.year, round(avg(run),2) as avgRun from car_records as cr, car_names as cn where cr.name_id = cn.name_id group by cn.name_id, cr.year")
rows = cursor.fetchall()

for row in rows:
    print(row)
    
cursor.close()
conn.close()

('Хонда Фит Шаттл', 2011, 167000.0)
('Хонда Фит Шаттл', 2013, 117000.0)
('Тойота Аллион', 2007, 290000.0)
('Тойота Аллион', 2018, 136000.0)
('Тойота Филдер', 2008, 182963.0)
('Тойота Филдер', 2009, 136000.0)
('Сузуки Солио', 2018, 99000.0)
('Тойота Альфард', 2019, 99000.0)
('Тойота Альфард', 2020, 90000.0)
('Тойота Альфард', 2021, 18000.0)
('Тойота Вокси', 2021, 28000.0)
('Тойота Харриер', 2014, 191000.0)
('Тойота Харриер', 2017, 95000.0)
('Тойота Харриер', 2018, 99000.0)
('Тойота Харриер', 2019, 55000.0)
('Тойота Харриер', 2020, 32000.0)
('Тойота Харриер', 2021, 32000.0)
('Тойота Харриер', 2023, 7860.0)
('Тойота Ленд Крузер Прадо', 2005, 206000.0)
('Тойота Ленд Крузер Прадо', 2011, 104000.0)
('Тойота Ленд Крузер Прадо', 2018, 73000.0)
('Тойота Ленд Крузер Прадо', 2020, 27250.0)
('Тойота Ленд Крузер Прадо', 2021, 30500.0)
('Тойота Ленд Крузер Прадо', 2022, 33000.0)
('Хонда Инсайт', 0, 120000.0)
('Хонда Инсайт', 2009, 145500.0)
('Дайхатсу Танто', 2017, 106000.0)
('Дайхатсу Танто', 2019,